# Classification Machine Learning Algorithm


### Classification-Neural-Network
Questions:
What is in the audio books dataset?
what did you set out to do with the dataset?
Did you achieve your initial goal?
How did neural nettwork help with your task?
why do you think neural nettwork help with your task?
why do you think neural network was the best choice for this task?

Data Description:
This is a data of audio books store which only sells the audio book online. Each person in the data set has atleast purchased the audio once. Our objective is to predict whether the customer will buy the audiobook again or not. \
The variables(column names) are
ID : each customer has its own ID,\
Book_length_min_overall: purchase: sum of length of purchase,\
Book_length_min_average: average sum of length of purchase,\
Price_overall: same like above two fields,\
,Price_avg: same as above description,\
Review: review given by customer or not if given 1 else 0,\
Review on 10/10: if review is 1 then the reviewed value for review 0 we take an average of review value and we fill up the shell for empty review,\
Minutes_listened: total listened minutes,\
Completion:it is the ratio of total_minutes_listened and book length overall,\
Support_request: it is whether user ask for support or not?,\
Last_visited_minus_purchase_date: The bigger the difference the better the user's engagement,\
and Targets: this is the targets(output in boolean value)\

The input data represents the two year of people engagements. We are doing the supervised learning so we need targets; tagrets 1 if a person converted, targets 0 if a person didn't converted\
Now the extra six months of data after the two  year period are also added to check if a user were converted or not. .i.e if a person bought another book or not. if they are converted then target is 1 else it is zero
So now we are creating a ML algorith that can predict whether a customer will buy a book again or not.
So this is a classification problem with two class won't buy and will buy represented by zeros and one\
\
Note: our inputs are all the variables of the csv file except for the first and the last column .i.e ID and Target\

For data preprocessing we have used these three proccesses:
* Balance the dataset
* Divide the data set into training validation and test
* Saving the data in tensorflow friendly format

we have used sklearn capabilities for standardizing the inputs by doing this we have gain the accuracy by 10%


### Extracting data from csv

In [1]:
import numpy as np
# we use sklern for standardization for higher accuracy
from sklearn import preprocessing

In [2]:
# loading except for last and first variables
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

# input except first and last column
unscaled_inputs_all = raw_csv_data[:,1:-1]

# last column of csv
targets_all = raw_csv_data[:,-1]

### Balancing the dataset
Since there are many items for target = 0 so we need to balance the dataset
* We will count the number of targets that are 1s
* We will keep as many 0s as 1s(we have deleted others)

In [8]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

### Standardized(Scale) the input

In [53]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Suffel the data
keeping the same information in different order

In [54]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Spliting the data for training,validating and testing
I'm using 0.80,0.10,0.10 ratio split


In [55]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1794.0 3579 0.501257334450964
227.0 447 0.5078299776286354
216.0 448 0.48214285714285715


### tensorflow friendly dataset
saving in .npz format

In [56]:
np.savez('Audiobooks_data_train',inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test',inputs = test_inputs, targets = test_targets)

We have completed the data preprocess competely for two class data.\
This code can be  generalized for all type of data preprocessing for any dataset with two classes \
Only we need two play around with balancing section if we have more classes

![](imageee.png)

Input is given from csv file and we have two possible outcome .i.e target =1 or target = 0\
we are assuming two hidden layer of 50 units on each hidden layer
![](img.png)

* Figure Description
The above figure is the network of our model. There are 10 inputs and only 2 outputs. \
Input data is taken from csv file and for two output; one is for target 1 and second is for target 0. In the dataset 1 and 0 are the two types of tems for the classification.


## Creating an Machine Learning Algorithm

### Importing the libraries

In [57]:
import numpy as np
import tensorflow as tf

### Loading the data

In [58]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

### Model
outline, optimizer, loss, early stopping and training

In [59]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          validation_data = (validation_inputs,validation_targets),
          verbose = 2
         )


Epoch 1/100
36/36 - 1s - loss: 0.5618 - accuracy: 0.7868 - val_loss: 0.4206 - val_accuracy: 0.8881
Epoch 2/100
36/36 - 0s - loss: 0.3702 - accuracy: 0.8757 - val_loss: 0.3026 - val_accuracy: 0.8971
Epoch 3/100
36/36 - 0s - loss: 0.3190 - accuracy: 0.8849 - val_loss: 0.2735 - val_accuracy: 0.9105
Epoch 4/100
36/36 - 0s - loss: 0.2994 - accuracy: 0.8902 - val_loss: 0.2649 - val_accuracy: 0.9150
Epoch 5/100
36/36 - 0s - loss: 0.2864 - accuracy: 0.8938 - val_loss: 0.2460 - val_accuracy: 0.9128
Epoch 6/100
36/36 - 0s - loss: 0.2747 - accuracy: 0.8977 - val_loss: 0.2411 - val_accuracy: 0.9217
Epoch 7/100
36/36 - 0s - loss: 0.2664 - accuracy: 0.9003 - val_loss: 0.2351 - val_accuracy: 0.9217
Epoch 8/100
36/36 - 0s - loss: 0.2588 - accuracy: 0.9019 - val_loss: 0.2251 - val_accuracy: 0.9239
Epoch 9/100
36/36 - 0s - loss: 0.2562 - accuracy: 0.9033 - val_loss: 0.2248 - val_accuracy: 0.9217
Epoch 10/100
36/36 - 0s - loss: 0.2502 - accuracy: 0.9056 - val_loss: 0.2263 - val_accuracy: 0.9239
Epoch 11/

36/36 - 0s - loss: 0.2068 - accuracy: 0.9218 - val_loss: 0.2221 - val_accuracy: 0.9217
Epoch 84/100
36/36 - 0s - loss: 0.2133 - accuracy: 0.9190 - val_loss: 0.2165 - val_accuracy: 0.9329
Epoch 85/100
36/36 - 0s - loss: 0.2060 - accuracy: 0.9226 - val_loss: 0.2063 - val_accuracy: 0.9329
Epoch 86/100
36/36 - 0s - loss: 0.2029 - accuracy: 0.9237 - val_loss: 0.2182 - val_accuracy: 0.9306
Epoch 87/100
36/36 - 0s - loss: 0.2049 - accuracy: 0.9237 - val_loss: 0.2161 - val_accuracy: 0.9329
Epoch 88/100
36/36 - 0s - loss: 0.2042 - accuracy: 0.9223 - val_loss: 0.2070 - val_accuracy: 0.9351
Epoch 89/100
36/36 - 0s - loss: 0.2069 - accuracy: 0.9234 - val_loss: 0.2063 - val_accuracy: 0.9329
Epoch 90/100
36/36 - 0s - loss: 0.2036 - accuracy: 0.9240 - val_loss: 0.2182 - val_accuracy: 0.9306
Epoch 91/100
36/36 - 0s - loss: 0.2051 - accuracy: 0.9232 - val_loss: 0.2060 - val_accuracy: 0.9329
Epoch 92/100
36/36 - 0s - loss: 0.2053 - accuracy: 0.9226 - val_loss: 0.2189 - val_accuracy: 0.9284
Epoch 93/100


High Accuracy so chance of overfitting \
Looking into the val_loss it gets increase somewhere between so no constant 
We have overfitted our model

### Testing the model

In [61]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 6ms/step - loss: 0.2862 - accuracy: 0.8929


In [62]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.29. Test accuracy: 89.29%


Using the initial model and hyperparameters , the final test accuracy should be roughly around 89%.

Each time the code is rerun, we get a different accuracy because each training is different.

We have intentionally reached a suboptimal solution, so we can have space to build on it!
